<a href="https://colab.research.google.com/github/IO-2024/IOTs_with_python/blob/main/Ch5.%20%EC%82%B0%EC%97%85%EC%97%B0%EA%B4%80%ED%9A%A8%EA%B3%BC%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

3부문 총, 국산, 수입거래표 및 고용표 불러오기

In [19]:
url_Tt_3 = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/Total_matrix_3sectors.csv"  # 3부문의 총거래표 불러오기
Tt_3 = pd.read_csv(url_Tt_3)

url_Dt_3 = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/Domestic_matrix_3sectors.csv"  # 3부문의 국산거래표 불러오기
Dt_3 = pd.read_csv(url_Dt_3)

url_It_3 = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/Imported_matrix_3sectors.csv"  # 3부문의 수입거래표 불러오기
It_3 = pd.read_csv(url_It_3)

url_emp_3 = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/Emp_3_secs.csv"  # 3부문의 고용표 불러오기
Emp_3 = pd.read_csv(url_emp_3)

In [20]:
Tt = Tt_3.to_numpy()  # numpy로 불러오기
Dt = Dt_3.to_numpy()
It = It_3.to_numpy()
Em = Emp_3.to_numpy()

In [21]:
Tt_1 = Tt[0:6,1:13].astype(float)
Dm_1 = Dt[0:4,1:10].astype(float)
Im_1 = It[0:4,1:10].astype(float)
Ep_1 = Em[:,1:3]

In [22]:
Ct = Tt_1[-1,0:3]  # 총산출액 벡터
Va = Tt_1[-2,0:3]  # 부가가치 벡터

In [23]:
Ct_m = np.tile(Ct, reps=[3,1]) # 총산출액 벡터를 3번 행반복

In [24]:
At = (Tt_1[0:3,0:3]/Ct_m)   # 총투입계수
Ad = (Dm_1[0:3,0:3]/Ct_m)   # 국산투입계수
Ai = (Im_1[0:3,0:3]/Ct_m)   # 수입투입계수
Av = (Va/Ct).reshape([1,3]) # 부가가치율 벡터

print('총투입계수표 \n', At.round(3),'\n')
print('국산투입계수표 \n', Ad.round(3),'\n')
print('수입투입계수표 \n', Ai.round(3),'\n')
print('부가가치율 \n', Av.round(3))

df_At = pd.DataFrame(At, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ad = pd.DataFrame(Ad, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ai = pd.DataFrame(Ai, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Av = pd.DataFrame(Av, columns = ['농산품', '공산품', '서비스'], index=['부가가치율'])                  #pandas로 내보내기

총투입계수표 
 [[0.067 0.021 0.004]
 [0.267 0.527 0.164]
 [0.117 0.143 0.283]] 

국산투입계수표 
 [[0.067 0.017 0.004]
 [0.25  0.328 0.132]
 [0.117 0.136 0.263]] 

수입투입계수표 
 [[0.    0.004 0.   ]
 [0.017 0.199 0.032]
 [0.    0.008 0.02 ]] 

부가가치율 
 [[0.55  0.309 0.548]]


##### 생산유발계수(Lf)

In [25]:
Lf = np.linalg.inv(np.identity(3)-Ad)  # 생산유발계수

Lf_rw_sm = Lf.sum(axis=0)  # 생산유발계수의 행합
Lf_cl_sm = Lf.sum(axis=1)  # 생산유발계수의 열합

Lf_bwd = (Lf.sum(axis=0)/Lf_rw_sm.mean()).astype(float).reshape([1,3])  #영향력 계수
Lf_fwd = (Lf.sum(axis=1)/Lf_cl_sm.mean()).astype(float).reshape([1,3])  #감응도 계수

print('생산유발계수 \n', Lf.round(3),'\n')

print('생산유발계수의 행합 \n', Lf_rw_sm.round(3),'\n')
print('생산유발계수의 열합 \n', Lf_cl_sm.round(3),'\n')

print('부문별 영향력계수 \n', Lf_bwd.round(3),'\n')
print('부문별 감응도계수 \n', Lf_fwd.round(3),'\n')

생산유발계수 
 [[1.081 0.03  0.011]
 [0.452 1.557 0.281]
 [0.254 0.291 1.411]] 

생산유발계수의 행합 
 [1.787 1.878 1.702] 

생산유발계수의 열합 
 [1.122 2.289 1.956] 

부문별 영향력계수 
 [[0.999 1.05  0.952]] 

부문별 감응도계수 
 [[0.627 1.279 1.093]] 



In [26]:
df_Lf = pd.DataFrame(Lf.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

#### 수입유발계수(Li)

In [27]:
Li = (Ai@Lf).round(3)  #수입유발계수
print('수입유발계수 \n', Li)

수입유발계수 
 [[0.002 0.006 0.002]
 [0.116 0.319 0.101]
 [0.009 0.018 0.03 ]]


#### 부가가치유발계수(Lv)

In [28]:
Va_m = np.diagflat(Av)  # 부가가치율의 대각행렬
Va_m

array([[0.55      , 0.        , 0.        ],
       [0.        , 0.30859375, 0.        ],
       [0.        , 0.        , 0.54842664]])

In [29]:
Lv = (Va_m@Lf).round(3)  #부가가치유발계수
print('부가가치유발계수 \n', Lv)

부가가치유발계수 
 [[0.595 0.017 0.006]
 [0.139 0.48  0.087]
 [0.139 0.16  0.774]]


In [30]:
df_Lv = pd.DataFrame(Lv.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  # pandas로 내보내기

In [31]:
df_Li = pd.DataFrame(Li.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

### 취업유발계수

In [32]:
Ei = ((Ep_1[:3,1]/Ct)*10).astype(float).round(3 ) # 10억원당 취업자수
Le = (np.diag(Ei)@Lf).round(3)  # 취업유발계수
print('취업계수 \n', Ei, '\n')
print('취업유발계수 \n', Le)

취업계수 
 [21.167  2.165  7.932] 

취업유발계수 
 [[22.88   0.638  0.241]
 [ 0.979  3.37   0.607]
 [ 2.017  2.31  11.189]]


In [33]:
df_Le = pd.DataFrame(Le.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

In [34]:
# CSV파일로 결과 내보내기
df_At.to_csv('총투입계수.csv', encoding='utf_8-sig')
df_Ad.to_csv('국산투입계수.csv', encoding='utf_8-sig')
df_Ai.to_csv('수입투입계수.csv', encoding='utf_8-sig')

df_Lf.to_csv('생산유발계수.csv', encoding='utf_8-sig')
df_Lv.to_csv('부가가치유발계수.csv', encoding='utf_8-sig')
df_Li.to_csv('수입유발계수.csv', encoding='utf_8-sig')
df_Le.to_csv('취업유발계수.csv', encoding='utf_8-sig')